In [ ]:
ZSBN_df = final_df[final_df.LocationName=="ZSBN"].copy()
fig = go.Figure()
fig.add_scatter(x=ZSBN_df.index, y=ZSBN_df.CO2)
fig.add_vrect(x0="2017-10-22", x1="2017-10-25",fillcolor="red", opacity=0.1)
fig.add_vline(x="2017-10-24",line_color="red")
fig.show()

In [ ]:
## The choice of the sensors was based essencially on their altitude (they approximately have the same altitude)

fig = go.Figure()
fig.add_scatter(x=final_df[final_df.LocationName=="ZUE"].index,y=final_df[final_df.LocationName=="ZUE"].CO2,name="ZUE")
fig.add_scatter(x=final_df[final_df.LocationName=="ZSTA"].index,y=final_df[final_df.LocationName=="ZSTA"].CO2,name="ZSTA")
fig.add_scatter(x=ZSBN_df.index,y=ZSBN_df.CO2,name="ZSBN")


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error


X = ZSBN_df[["temperature","humidity"]]
y = ZSBN_df[["CO2"]]
fig = go.Figure()
fig.add_scatter(x=ZSBN['timestamp'], y=ZSBN['CO2'], mode='lines', name="Measurements")
fig.add_scatter(x=ZSBN['timestamp'], y=ZSBN['prediction'], mode='lines', name="Prediction")
fig.update_xaxes(
    dtick=24*60*60*1000
    #,ticklabelmode='period'
)
fig.update_layout(
    title='CO2 Level (ppm) in October 2017',
    xaxis_title='Date',
    yaxis_title='CO2 (ppm)',
    hovermode='x unified'
)
fig.show()

##CROSS_VALIDATION
##Fama mochkla fel cross_validation lezem nthabet fiha
K = 10
tscv = TimeSeriesSplit(K)

rmse = []
scores = []

cv_pred = []
cv_measurements = []
train_times=[]    
test_times=[]
for i, (train_index, test_index) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    reg = LinearRegression().fit(X_train,y_train)
    y_pred = reg.predict(X_test)
    
    cv_pred.append(y_pred)
    
    cv_measurements.append(y_test)

    scores.append( reg.score(X_test,y_test) )
    rmse.append( np.sqrt(2 *mean_squared_error(y_test, y_pred)) )
    ##train_times.append([X_train.reset_index()['timestamp'].to_list()[0],X_train.reset_index()['timestamp'].to_list()[-1]])
    ##test_times.append([X_test.reset_index()['timestamp'].to_list()[0],X_test.reset_index()['timestamp'].to_list()[-1]])
    train_times.append(X_train.reset_index()['timestamp'].to_list())
    test_times.append(X_test.reset_index()['timestamp'].to_list())